## units only ftrs : camera remote/local (eg. away/home) center moves

#### Taehwan jeong (xikizima@hotmail.com)

### 공통(Common)

In [1]:
import pickle, datetime
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_colwidth = -1

from collections import Counter

from matplotlib import pyplot as plt

In [2]:
# 정규식
import re
cmp = re.compile('(?:Target:\s([A-Za-z]*)\s\[([0-9A-Z]*)\][\s;]*)?Location:\s\(([0-9]*)[.0-9]*,\s([0-9]*)[.0-9]*.*?\)')
coors = re.compile('(?:Location:|at)\s\(([0-9]*)[.0-9]*,\s([0-9]*)[.0-9]*.*?\)')
units = re.compile('([0-9A-Za-z]*)\s\[([0-9A-Z]*)\]')

### 학습용 (Train)

In [3]:
def build_camera_center_moves_ftr(df, idx):
    df = df[(df['game_id'] < (idx*1000)+1000) & (df['game_id'] >= (idx*1000))]
    df = df[df['event'].isin(['Camera', 'Right Click'])]

    arr_coors = df['event_contents'].map(coors.findall).map(lambda x: x[0]).apply(pd.Series)
    arr_coors.columns = ['x', 'y']
    arr_coors['x'] = arr_coors['x'].astype('int')
    arr_coors['y'] = arr_coors['y'].astype('int')
    df = pd.concat([df, arr_coors], axis=1)

    df.loc[:, 'slice'] = (df['time'] // 1).astype(int)
    df = df[df['slice'] < 10]

    center_player = df.groupby(by=['game_id', 'player']).first()[['x', 'y']]

    df = df.groupby(by=['game_id', 'player','slice'])[['x', 'y']].mean().reset_index()
    df = pd.merge(df, center_player, left_on=['game_id', 'player'], right_index=True, suffixes=['', '_center'])
    df['dist_from_base'] = np.sqrt((df['x'] - df['x_center']) ** 2 + (df['y'] - df['y_center']) ** 2)

    center_moves_p0 = df[df['player'] == 0].set_index(['game_id', 'player', 'slice'])['dist_from_base'].unstack('slice')
    center_moves_p0.columns = [f'p0_center_{idx}' for idx in center_moves_p0.columns]
    center_moves_p0 = center_moves_p0.droplevel(1).reset_index()
    center_moves_p1 = df[df['player'] == 1].set_index(['game_id', 'player', 'slice'])['dist_from_base'].unstack('slice')
    center_moves_p1.columns = [f'p1_center_{idx}' for idx in center_moves_p1.columns]
    center_moves_p1 = center_moves_p1.droplevel(1).reset_index()
    ret = pd.merge(center_moves_p0, center_moves_p1, on='game_id').fillna(0.0)
    return ret

In [4]:
df = pd.read_csv('./data/train.csv') #, nrows=4000000)
for idx in range(39):
    t1 = datetime.datetime.now()
    ret = build_camera_center_moves_ftr(df, idx)
    ret.to_csv('./data/camera_center_moves_train.csv', mode='a' if idx > 0 else 'w', index=False, header=None if idx > 0 else True)
    print(f'[{idx}] {(datetime.datetime.now() - t1).seconds:.1f} secs')

[0] 247.0 secs
[1] 241.0 secs
[2] 252.0 secs
[3] 238.0 secs
[4] 241.0 secs
[5] 241.0 secs
[6] 240.0 secs
[7] 246.0 secs
[8] 245.0 secs
[9] 243.0 secs
[10] 241.0 secs
[11] 242.0 secs
[12] 244.0 secs
[13] 240.0 secs
[14] 242.0 secs
[15] 238.0 secs
[16] 242.0 secs
[17] 246.0 secs
[18] 237.0 secs
[19] 240.0 secs
[20] 247.0 secs
[21] 248.0 secs
[22] 245.0 secs
[23] 242.0 secs
[24] 244.0 secs
[25] 240.0 secs
[26] 248.0 secs
[27] 239.0 secs
[28] 242.0 secs
[29] 244.0 secs
[30] 246.0 secs
[31] 246.0 secs
[32] 243.0 secs
[33] 242.0 secs
[34] 248.0 secs
[35] 242.0 secs
[36] 247.0 secs
[37] 255.0 secs
[38] 209.0 secs


### 검증용 (Test)

In [5]:
df = pd.read_csv('./data/test.csv')
START_IDX = 38
for idx in range(START_IDX, 56):
    t1 = datetime.datetime.now()
    ret = build_camera_center_moves_ftr(df, idx)
    ret.to_csv('./data/camera_center_moves_test.csv', mode='a' if idx > START_IDX else 'w', index=False, header=None if idx > START_IDX else True)
    print(f'[{idx}] {(datetime.datetime.now() - t1).seconds:.1f} secs')

[38] 32.0 secs
[39] 240.0 secs
[40] 236.0 secs
[41] 236.0 secs
[42] 232.0 secs
[43] 234.0 secs
[44] 231.0 secs
[45] 233.0 secs
[46] 237.0 secs
[47] 244.0 secs
[48] 232.0 secs
[49] 228.0 secs
[50] 236.0 secs
[51] 239.0 secs
[52] 242.0 secs
[53] 234.0 secs
[54] 238.0 secs
[55] 159.0 secs
